In [219]:
using CSV, DataFrames, Plots;
plotly();

In [220]:
df = CSV.read("mammographic_masses.data", null="?", rows_for_type_detect = 444)
df = dropmissing(df)

head(df)

,bi-rads,age,shape,margin,density,severity
1,5,67,3,5,3,1
2,5,58,4,5,3,1
3,4,28,1,1,3,0
4,5,57,1,5,3,1
5,5,76,1,4,3,1
6,3,42,2,1,3,1


In [221]:
X = Array(df[:, 1:5])
y = df[:, 6]

μ = mean(X, 1)
σ = std(X, 1)

m, n = size(X)

X = (X .- μ) ./ σ;

X = hcat(zeros(m), X)
n += 1;

X_test = X[1:Int(m*0.2), :];
X = X[Int(m*0.2)+1:end, :];

y_test = y[1:Int(m*0.2), :];
y = y[Int(m*0.2)+1:end, :];

In [229]:
function sigmoid(z)
    1.0 ./ (1.0 .+ exp.(-z))
end

θ = ones(n)
threshold = 0.000002
α = 0.001;
past_cost = zeros(0)
λ = 3

function cost(current_θ)
    h = sigmoid(sum(transpose(current_θ) .* X, 2));
    -1/m * (sum(y .* log.(h) + (1-y).*(log.(1-h))) + λ * sum(θ.^2))
end

@time while true
    h = sigmoid(sum(transpose(θ) .* X, 2))
    θ = θ - transpose(α/m * sum((h - y).* X, 1))
    append!(past_cost, cost(θ))
    if length(past_cost) > 1 && past_cost[end-1] - past_cost[end] <= threshold
        break
    end
end

count = 0
same = y_test .== round.(sigmoid(sum(transpose(θ) .* X_test, 2)))
length(same[same])/length(y_test)

  3.257828 seconds (3.12 M allocations: 1.942 GiB, 10.58% gc time)


0.7951807228915663

In [230]:
plot(past_cost)